In [1]:
# Dependencies and Setup
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy.stats as st
from scipy.stats import linregress

# Team data files
hockey_team_path = "hockey_starting_data/Teams.csv"

# Read the yearly team data and results
hockey_team_data = pd.read_csv(hockey_team_path)


# Display the data table for preview
hockey_team_data.head()

,year,lgID,tmID,franchID,confID,divID,rank,playoff,G,W,...,GA,name,PIM,BenchMinor,PPG,PPC,SHA,PKG,PKC,SHF
0,1909,NHA,COB,BKN,NaN,NaN,4,NaN,12,4,...,104,Cobalt Silver Kings,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1909,NHA,HAI,MTL,NaN,NaN,5,NaN,12,4,...,83,Haileybury Hockey Club,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1909,NHA,LES,TBS,NaN,NaN,7,NaN,12,2,...,100,Les Canadiens,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1909,NHA,MOS,MOS,NaN,NaN,6,NaN,12,3,...,95,Montreal Shamrocks,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1909,NHA,MOW,MTW,NaN,NaN,1,NaN,12,11,...,41,Montreal Wanderers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
# The original dataframe included teams from leagues outside of the National Hockey League (NHL), so we are filtering for just NHL data
nhl_team_data_uncleaned = pd.DataFrame(hockey_team_data.loc[(hockey_team_data["lgID"]=="NHL"),:])

# A lockout in 1994 forced only a portion of the season to be played, so we will delete that year's data to eliminate future outliers
nhl_team_data = pd.DataFrame(nhl_team_data_uncleaned.loc[(nhl_team_data_uncleaned["year"]!=1994),:])
nhl_team_data.head()

,year,lgID,tmID,franchID,confID,divID,rank,playoff,G,W,...,GA,name,PIM,BenchMinor,PPG,PPC,SHA,PKG,PKC,SHF
65,1917,NHL,MTL,MTL,NaN,NaN,1,NHLF,22,13,...,84,Montreal Canadiens,336.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
66,1917,NHL,MTW,MTW,NaN,NaN,4,NaN,6,1,...,35,Montreal Wanderers,27.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
67,1917,NHL,OTS,STE,NaN,NaN,3,NaN,22,9,...,114,Ottawa Senators,333.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
68,1917,NHL,TOA,TOR,NaN,NaN,1,SC,22,13,...,109,Toronto Arenas,355.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
72,1918,NHL,MTL,MTL,NaN,NaN,2,ND,18,10,...,78,Montreal Canadiens,253.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# Created a new DataFrame that displays each team's first value
nhl_team_data
nhl_team_sort = nhl_team_data.sort_values("year", ascending=True)
nhl_team_sort = nhl_team_sort.drop_duplicates("tmID", keep='first')
nhl_team_sort.head()

,year,lgID,tmID,franchID,confID,divID,rank,playoff,G,W,...,GA,name,PIM,BenchMinor,PPG,PPC,SHA,PKG,PKC,SHF
65,1917,NHL,MTL,MTL,NaN,NaN,1,NHLF,22,13,...,84,Montreal Canadiens,336.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
66,1917,NHL,MTW,MTW,NaN,NaN,4,NaN,6,1,...,35,Montreal Wanderers,27.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
67,1917,NHL,OTS,STE,NaN,NaN,3,NaN,22,9,...,114,Ottawa Senators,333.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
68,1917,NHL,TOA,TOR,NaN,NaN,1,SC,22,13,...,109,Toronto Arenas,355.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80,1919,NHL,QUB,BKN,NaN,NaN,4,NaN,24,4,...,177,Quebec Bulldogs,123.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
nhl_teams = nhl_team_sort.loc[:, ["year", "lgID", "tmID", "franchID", "name"]]
nhl_teams

,year,lgID,tmID,franchID,name
65,1917,NHL,MTL,MTL,Montreal Canadiens
66,1917,NHL,MTW,MTW,Montreal Wanderers
67,1917,NHL,OTS,STE,Ottawa Senators
68,1917,NHL,TOA,TOR,Toronto Arenas
80,1919,NHL,QUB,BKN,Quebec Bulldogs
81,1919,NHL,TRS,TOR,Toronto St. Patricks
85,1920,NHL,HAM,BKN,Hamilton Tigers
125,1924,NHL,BOS,BOS,Boston Bruins
128,1924,NHL,MTM,MTM,Montreal Maroons
142,1925,NHL,PIP,PHQ,Pittsburgh Pirates


In [7]:
nhl_teams["franchID"].nunique()

35

In [6]:
nhl_teams.to_csv('nhl_team_founding_years.csv', index=False)